In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
params = { 'axes.grid': True
          ,'axes.axisbelow': True
          ,'grid.color': '0.5'
          ,'legend.numpoints': 1
          ,'savefig.bbox': 'tight'
          ,'font.size': 16
          ,'legend.loc': 'best'
          ,'legend.fontsize': 'medium'
          #,'lines.markersize': 6
          ,'xtick.labelsize': 'small'
          ,'ytick.labelsize': 'small'
          #,'xtick.direction': 'out'
          #,'ytick.direction': 'out'
          #,'text.usetex': False
          ,'figure.figsize': [8, 6]
          #,'axes.labelsize': 'large'   # no need to change these sizes, just change font.size
          #,'axes.titlesize': 'x-large'
          ,'axes.formatter.useoffset': False
}
plt.rcParams.update(params)

# Get county data

In [3]:
County_info_df= pd.read_csv('../county_info.csv')

In [4]:
County_info_df.columns

Index([u'County', u'FIPS'], dtype='object')

In [6]:
print 'Number of counties: %d' % len(County_info_df)
print ''
print County_info_df['County']

Number of counties: 58

0             Alameda
1              Alpine
2              Amador
3               Butte
4           Calaveras
5              Colusa
6        Contra Costa
7           Del Norte
8           El Dorado
9              Fresno
10              Glenn
11           Humboldt
12           Imperial
13               Inyo
14               Kern
15              Kings
16               Lake
17             Lassen
18        Los Angeles
19             Madera
20              Marin
21           Mariposa
22          Mendocino
23             Merced
24              Modoc
25               Mono
26           Monterey
27               Napa
28             Nevada
29             Orange
30             Placer
31             Plumas
32          Riverside
33         Sacramento
34         San Benito
35     San Bernardino
36          San Diego
37      San Francisco
38        San Joaquin
39    San Luis Obispo
40          San Mateo
41      Santa Barbara
42        Santa Clara
43         Santa Cruz
44      

# Check the ED visits data

In [7]:
ED_visits_df= pd.read_csv('ASED_0514.csv')

In [8]:
ED_visits_df.columns

Index([u'FIPS', u'Year', u'Value', u'Value Type', u'Ethnicity', u'Age',
       u'Gender'],
      dtype='object')

In [9]:
ED_visits_df.describe()

,FIPS,Year,Value
count,142200.000000,142200.000000,142194.000000
mean,6058.954149,2009.521435,261.302678
std,33.188348,2.878453,2389.383971
min,6000.000000,2005.000000,1.540000
25%,6031.000000,2007.000000,33.870000
50%,6065.000000,2010.000000,58.460000
75%,6085.000000,2012.000000,121.000000
max,6115.000000,2014.000000,186540.000000


In [10]:
print ED_visits_df['FIPS'].sort_values().unique()
print len(ED_visits_df['FIPS'].unique())

[6000 6001 6005 6007 6009 6011 6013 6015 6017 6019 6021 6023 6025 6027 6029
 6031 6033 6035 6037 6039 6041 6043 6045 6047 6049 6051 6053 6055 6057 6059
 6061 6063 6065 6067 6069 6071 6073 6075 6077 6079 6081 6083 6085 6087 6089
 6091 6093 6095 6097 6099 6101 6103 6105 6107 6109 6111 6113 6115]
58


We have a code for California (6000) but no code for Alpine (6003)

In [11]:
ED_visits_df= ED_visits_df[ED_visits_df['FIPS']!=6000]

Get only kids with ages bellow 18

In [12]:
ED_visits_df['Age'].unique()

array(['0004', '0517', '1834', '3564', '65OV', 'TOTL'], dtype=object)

In [13]:
ED_visits_df= ED_visits_df[(ED_visits_df['Age']=='0004') | (ED_visits_df['Age']=='0517')]

In [14]:
ED_visits_df['Age'].unique()

array(['0004', '0517'], dtype=object)

Let's separate the number and rates into different columns

In [15]:
ED_visits_df['Value Type'].unique()

array(['N', 'R10K', 'R10KLL', 'R10KUL'], dtype=object)

In [16]:
number_ED_visits_df= ED_visits_df[ED_visits_df['Value Type']=='N']
number_ED_visits_df= number_ED_visits_df.rename(columns={'Value':'Number'})
number_ED_visits_df= number_ED_visits_df.drop('Value Type', axis=1)
print len(number_ED_visits_df)

10584


In [17]:
number_ED_visits_df.head()

,FIPS,Year,Number,Ethnicity,Age,Gender
4320,6001,2005,257.0,AFAM,0004,FEML
4321,6001,2005,420.0,AFAM,0004,MALE
4322,6001,2005,677.0,AFAM,0004,TOTL
4323,6001,2005,406.0,AFAM,0517,FEML
4324,6001,2005,461.0,AFAM,0517,MALE


In [18]:
rates_ED_visits_df= ED_visits_df[ED_visits_df['Value Type']=='R10K']
rates_ED_visits_df= rates_ED_visits_df.rename(columns={'Value':'Rates'})
rates_ED_visits_df= rates_ED_visits_df.drop('Value Type', axis=1)
print len(rates_ED_visits_df)

10584


In [19]:
rates_ED_visits_df.head()

,FIPS,Year,Rates,Ethnicity,Age,Gender
4428,6001,2005,448.28,AFAM,0004,FEML
4429,6001,2005,708.98,AFAM,0004,MALE
4430,6001,2005,580.77,AFAM,0004,TOTL
4431,6001,2005,221.79,AFAM,0517,FEML
4432,6001,2005,254.14,AFAM,0517,MALE


Join the databases

In [20]:
ED_df= pd.merge(number_ED_visits_df,rates_ED_visits_df,how='inner',on=['FIPS','Year','Ethnicity','Age','Gender'])
print len(ED_df)

10584


In [21]:
ED_df.head()

,FIPS,Year,Number,Ethnicity,Age,Gender,Rates
0,6001,2005,257.0,AFAM,0004,FEML,448.28
1,6001,2005,420.0,AFAM,0004,MALE,708.98
2,6001,2005,677.0,AFAM,0004,TOTL,580.77
3,6001,2005,406.0,AFAM,0517,FEML,221.79
4,6001,2005,461.0,AFAM,0517,MALE,254.14


Drop all rows with totals

In [22]:
ED_df= ED_df[ED_df['Gender']!='TOTL']

In [23]:
ED_df= ED_df[ED_df['Ethnicity']!='TOTL']

In [24]:
# Confirming there's no NaN values
ED_df.isnull().any(0)

FIPS         False
Year         False
Number       False
Ethnicity    False
Age          False
Gender       False
Rates        False
dtype: bool

Create number variables for the category variables

In [25]:
ED_df['Age (<4)']= [0]*len(ED_df)

ED_df['Gender (M)']= [0]*len(ED_df)

ED_df['White']= [0]*len(ED_df)
ED_df['African']= [0]*len(ED_df)
ED_df['Asian']= [0]*len(ED_df)
ED_df['Hispanic']= [0]*len(ED_df)
ED_df['Other Ethnicity']= [0]*len(ED_df)

In [26]:
ED_df.loc[ED_df['Age']=='0004','Age (<4)']= 1

In [27]:
ED_df.loc[ED_df['Gender']=='MALE','Gender (M)']= 1

In [28]:
ED_df.loc[ED_df['Ethnicity']=='WHIT','White']= 1
ED_df.loc[ED_df['Ethnicity']=='AFAM','African']= 1
ED_df.loc[ED_df['Ethnicity']=='ASPI','Asian']= 1
ED_df.loc[ED_df['Ethnicity']=='HISP','Hispanic']= 1
ED_df.loc[ED_df['Ethnicity']=='OTHR','Other Ethnicity']= 1

In [29]:
ED_df.head()

,FIPS,Year,Number,Ethnicity,Age,Gender,Rates,Age (<4),Gender (M),White,African,Asian,Hispanic,Other Ethnicity
0,6001,2005,257.0,AFAM,0004,FEML,448.28,1,0,0,1,0,0,0
1,6001,2005,420.0,AFAM,0004,MALE,708.98,1,1,0,1,0,0,0
3,6001,2005,406.0,AFAM,0517,FEML,221.79,0,0,0,1,0,0,0
4,6001,2005,461.0,AFAM,0517,MALE,254.14,0,1,0,1,0,0,0
6,6001,2005,61.0,ASPI,0004,FEML,51.37,1,0,0,0,1,0,0


Drop category variables

In [30]:
ED_df= ED_df.drop('Ethnicity', axis=1)
ED_df= ED_df.drop('Gender', axis=1)
ED_df= ED_df.drop('Age', axis=1)

In [31]:
ED_df.head()

,FIPS,Year,Number,Rates,Age (<4),Gender (M),White,African,Asian,Hispanic,Other Ethnicity
0,6001,2005,257.0,448.28,1,0,0,1,0,0,0
1,6001,2005,420.0,708.98,1,1,0,1,0,0,0
3,6001,2005,406.0,221.79,0,0,0,1,0,0,0
4,6001,2005,461.0,254.14,0,1,0,1,0,0,0
6,6001,2005,61.0,51.37,1,0,0,0,1,0,0


In [32]:
print ED_df['FIPS'].sort_values().unique()
print len(ED_df['FIPS'].unique())

[6001 6005 6007 6009 6011 6013 6015 6017 6019 6023 6025 6027 6029 6031 6033
 6035 6037 6039 6041 6045 6047 6053 6055 6057 6059 6061 6065 6067 6069 6071
 6073 6075 6077 6079 6081 6083 6085 6087 6089 6093 6095 6097 6099 6101 6103
 6107 6109 6111 6113 6115]
50


In [42]:
FIPS_array= ED_df['FIPS'].unique()
for f in County_info_df['FIPS']:
    if not(f in FIPS_array):
        print f, County_info_df.loc[County_info_df['FIPS']==f,'County']

6003 1    Alpine
Name: County, dtype: object
6021 10    Glenn
Name: County, dtype: object
6043 21    Mariposa
Name: County, dtype: object
6049 24    Modoc
Name: County, dtype: object
6051 25    Mono
Name: County, dtype: object
6063 31    Plumas
Name: County, dtype: object
6091 45    Sierra
Name: County, dtype: object
6105 52    Trinity
Name: County, dtype: object


Missing 8 counties: Alpine, Glenn, Mariposa, Modoc, Mono, Plumas, Sierra, Trinity

In [35]:
print 'Total number of data points: %d' % len(ED_df)

Total number of data points: 4840


Save corresponding file

In [138]:
ED_df.to_csv('temp_ED_visits.csv',index=False, header=True)